In [72]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
import os
#from pmdarima import auto_arima
import sys
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
import time
from tqdm.notebook import tqdm
from google.colab import files

# Setting Inputs

## Setting Dates

In [73]:
# date format yyyy-mm-dd

# the date from which we have data
known_start_dt = '2018-01-01' # decide
# the date till which we have data
known_end_dt = '2020-06-03' # decide
unknown_end_dt = '2020-06-30' # decide, generally year end

In [74]:
# original data
df_all = pd.read_csv('https://raw.githubusercontent.com/datacoe-publicissapient/risingai2020/master/data/Model_uncapped.csv')

# changing date to proper format
df_all.date = pd.to_datetime(df_all.date)

# setting date as index
df_all.set_index('date', inplace = True)

In [75]:
# outlier treated data
df_capped = pd.read_csv('https://raw.githubusercontent.com/datacoe-publicissapient/risingai2020/master/data/Model_Final.csv')

# changing date to proper format
df_capped.date = pd.to_datetime(df_capped.date)

# setting date as index
df_capped.set_index('date', inplace = True)

In [76]:
# pdq_visit_unpaid_decided = 'Final_Visits_PDQ.xlsx'
# pdq_leads_decided = 'Final_Leads_PDQ.xlsx'


pdq_visit_unpaid_decided = 'https://raw.githubusercontent.com/datacoe-publicissapient/risingai2020/master/data/Final_Visits_PDQ.csv'
pdq_leads_decided = 'https://raw.githubusercontent.com/datacoe-publicissapient/risingai2020/master/data/Final_Leads_PDQ.csv'


# Functions

In [77]:
#MAPE function
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

In [78]:
def alldates(start_dt, end_dt):
    '''
    input in 'yyyy-mm-dd' string format
    '''
    
    start_dt = datetime.strptime(start_dt, '%Y-%m-%d').date()
    end_dt = datetime.strptime(end_dt, '%Y-%m-%d').date()
    
    def daterange(date1, date2):
        for n in range(int ((date2 - date1).days)+1):
            yield date1 + timedelta(n)

    dt_list = list()
    for dt in daterange(start_dt, end_dt):
        #print(dt.strftime("%Y-%m-%d"))
        dt_list.append(dt.strftime("%Y-%m-%d"))
    
    return(dt_list)

## RMSE/MAPE

In [79]:
def rmse_mape_60(df_all,df_capped,pdq,var,exog_var,if_outlier_treat,nameplate,known_start_dt,known_end_dt,unknown_end_dt):
    start_time = time.time()

    # # Setting Dates based on input

    # last two months + some extra days of 3rd month of known data (if any)
    # currently taking as 60 days before known_end_dt 
    known_last2_start_dt = (datetime.strptime(known_end_dt, '%Y-%m-%d').date() - timedelta(days = 60)).strftime("%Y-%m-%d")


    # this is one day before known_last2_start_dt
    end_dt_before_last2 = (datetime.strptime(known_last2_start_dt, '%Y-%m-%d').date() - timedelta(days = 1)).strftime("%Y-%m-%d")

    # from and to date of range we want to predict
    # generally one day after known_end_dt
    unknown_start_dt = (datetime.strptime(known_end_dt, '%Y-%m-%d').date() + timedelta(days = 1)).strftime("%Y-%m-%d")


    # last 31 days of known data
    # 31 days before known_end_dt
    last31_start_dt = (datetime.strptime(known_end_dt, '%Y-%m-%d').date() - timedelta(days = 30)).strftime("%Y-%m-%d")

    
    # subsetting df_all for only the given nameplate
    df_a = df_all[df_all.Category == nameplate]
    df_c = df_capped[df_capped.Category == nameplate]
    
    
    # train on data till 60 days before last day of known data.
    # forecast on last 60 days of known data and calculate RMSE and MAPE
    
    train_last60_a = df_a[known_start_dt:end_dt_before_last2]
    test_last60_a = df_a[known_last2_start_dt:known_end_dt]
    
    train_last60_c = df_c[known_start_dt:end_dt_before_last2]
    
    
    # var   
    #----------creating pdq = 513 for given varialbe in var 
    try:
        # setting p d q values
        p = int(pdq[0])
        d = int(pdq[1])
        q = int(pdq[2])
        
        # deciding training data
        if if_outlier_treat == 'Y':
            train_last60 = train_last60_c.copy()
        elif if_outlier_treat == 'N':
            train_last60 = train_last60_a.copy()
        else:
            print(sys._getframe(  ).f_code.co_name+':error=if_outlier_treat flag wrong: if_outlier_treat= '+if_outlier_treat)
        
        # setting test data
        test_last60 = test_last60_a.copy()
        
        # ARIMAX model
        ar = ARIMA(endog=train_last60[var], exog=train_last60[exog_var], order=(p,d,q))
        # fit ARIMA model
        model = ar.fit()
        # predict test using ARIMA model
        pred_test_last60 = model.forecast(steps=len(test_last60), exog=test_last60[exog_var])
        
        # calculating MAPE & RMSE
        # also handling infinite MAPE incase of leads_total as it has 0 values too
        #if var == 'leads_total':
            ####################################################
            # handling inf in MAPE by removing actual total_leads == 0
        df_mape_inf = pd.DataFrame()
        df_mape_inf[var] = test_last60[var]
        df_mape_inf['preds'] = pred_test_last60[0]
        df_mape_inf = df_mape_inf[df_mape_inf[var] != 0]
            #############################################
        rmse = round(np.sqrt(mean_squared_error(test_last60[var], pred_test_last60[0])), 0)
        mape = round(mean_absolute_percentage_error(df_mape_inf[var], df_mape_inf['preds']), 2)
        #else:
            # calculating MAPE & RMSE
            #rmse = round(np.sqrt(mean_squared_error(test_last60[var], pred_test_last60[0])), 0)
            #mape = round(mean_absolute_percentage_error(test_last60[var], pred_test_last60[0]), 2)
        
    except Exception as e:
        print(sys._getframe(  ).f_code.co_name+':nameplate='+nameplate+':variable='+var+':pdq='+pdq+':if_outlier_treat='+if_outlier_treat+':error='+str(e))
        rmse = 'error'
        mape = 'error'
    
    end_time = time.time()
    print(sys._getframe(  ).f_code.co_name+':nameplate='+nameplate+':variable='+var+':pdq='+pdq+':if_outlier_treat='+if_outlier_treat+':Time Taken={0} seconds'.format(end_time - start_time))
    
    return rmse, mape

### RMSE/MAPE for 511 and no outlier treatment

In [80]:
start_time = time.time()


df_rmse_mape = pd.DataFrame(columns = ['Nameplate','Variable','pdq','if_outlier_treat','RMSE','MAPE'])



for nameplate in tqdm(df_all.Category.unique()):

        
    for var in ['visits','testdrive','leads']:
        
        if var in ['visits','testdrive']:
            exog_var = 'total_spend_nobrand_nodig'            
        elif var == 'leads':
            exog_var == 'total_spend_nobrand'
        else:
            print('error: wrong variable: var='+var)
            
        pdq = '511'
        if_outlier_treat = 'N'
    
        rmse, mape = rmse_mape_60(df_all,df_capped,pdq,var,exog_var,if_outlier_treat,nameplate,known_start_dt,known_end_dt,unknown_end_dt)
        df_rmse_mape = df_rmse_mape.append({'Nameplate':nameplate,
                                            'Variable':var,
                                            'pdq':pdq,
                                            'if_outlier_treat':if_outlier_treat,
                                            'RMSE':rmse,
                                            'MAPE':mape}, ignore_index = True)
    
    
end_time = time.time()
print("Time Taken RMSE/MAPE: {0} seconds".format(end_time - start_time))

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


rmse_mape_60:nameplate=Cat1:variable=visits:pdq=511:if_outlier_treat=N:Time Taken=4.0649261474609375 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


rmse_mape_60:nameplate=Cat1:variable=testdrive:pdq=511:if_outlier_treat=N:Time Taken=1.4136161804199219 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


...

rmse_mape_60:nameplate=Cat11:variable=visits:pdq=511:if_outlier_treat=N:Time Taken=2.8626012802124023 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


rmse_mape_60:nameplate=Cat11:variable=testdrive:pdq=511:if_outlier_treat=N:Time Taken=1.4517672061920166 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


rmse_mape_60:nameplate=Cat11:variable=leads:pdq=511:if_outlier_treat=N:Time Taken=0.8194334506988525 seconds

Time Taken RMSE/MAPE: 53.0224175453186 seconds


In [82]:
df_rmse_mape.to_csv('MAPE_RMSE_unpaid_511_noOUTLIERtreat.csv', index = False)

### RMSE/MAPE for suggested pdqs and outlier treatment

In [83]:
df_all.Category.unique()

array(['Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8',
       'Cat9', 'Cat10', 'Cat11'], dtype=object)

In [84]:
start_time = time.time()

# reading finally decided pdq and outlier treat
#pdq_map_vu = pd.read_excel(pdq_visit_unpaid_decided)
pdq_map_vu = pd.read_csv(pdq_visit_unpaid_decided)
pdq_map_vu['Nameplate'] = pdq_map_vu['Nameplate'].astype(str)
#pdq_map_l = pd.read_excel(pdq_leads_decided)
pdq_map_l = pd.read_csv(pdq_leads_decided)
pdq_map_l['Nameplate'] = pdq_map_l['Nameplate'].astype(str)


df_rmse_mape = pd.DataFrame(columns = ['Nameplate','Variable','pdq','if_outlier_treat','RMSE','MAPE'])



for nameplate in tqdm(df_all.Category.unique()):
        
    for var in ['visits','testdrive','leads']:
        
        if var in ['visits','testdrive','leads']:
            exog_var = 'total_spend_nobrand_nodig'
            
            pdq_map_vu_nameplate = pdq_map_vu[pdq_map_vu.Nameplate == nameplate]
            p = str(pdq_map_vu_nameplate['p'].values[0])
            d = str(pdq_map_vu_nameplate['d'].values[0])
            q = str(pdq_map_vu_nameplate['q'].values[0])
            pdq = p+d+q
            if_outlier_treat = str(pdq_map_vu_nameplate['outlier_treatment'].values[0])
            
        elif var == 'leads':
            exog_var == 'total_spend_nobrand'
            
            pdq_map_l_nameplate = pdq_map_l[pdq_map_l.Nameplate == nameplate]
            p = str(pdq_map_l_nameplate['p'].values[0])
            d = str(pdq_map_l_nameplate['d'].values[0])
            q = str(pdq_map_l_nameplate['q'].values[0])
            pdq = p+d+q
            if_outlier_treat = str(pdq_map_l_nameplate['outlier_treatment'].values[0])
            
        else:
            print('error: wrong variable: var='+var)
            
    
        rmse, mape = rmse_mape_60(df_all,df_capped,pdq,var,exog_var,if_outlier_treat,nameplate,known_start_dt,known_end_dt,unknown_end_dt)
        df_rmse_mape = df_rmse_mape.append({'Nameplate':nameplate,
                                            'Variable':var,
                                            'pdq':pdq,
                                            'if_outlier_treat':if_outlier_treat,
                                            'RMSE':rmse,
                                            'MAPE':mape}, ignore_index = True)
    
    
end_time = time.time()
print("Time Taken RMSE/MAPE: {0} seconds".format(end_time - start_time))

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored wh

rmse_mape_60:nameplate=Cat1:variable=visits:pdq=514:if_outlier_treat=N:error=The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
rmse_mape_60:nameplate=Cat1:variable=visits:pdq=514:if_outlier_treat=N:Time Taken=0.08277297019958496 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


rmse_mape_60:nameplate=Cat1:variable=testdrive:pdq=514:if_outlier_treat=N:Time Taken=13.652445554733276 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


...

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWa

rmse_mape_60:nameplate=Cat11:variable=visits:pdq=514:if_outlier_treat=N:Time Taken=12.140738248825073 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/tsatools.py:668: RuntimeWarning: overflow encountered in exp
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/tsatools.py:668: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/tsatools.py:669: RuntimeWarning: overflow encountered in exp
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/tsatools.py:669: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_mod

rmse_mape_60:nameplate=Cat11:variable=testdrive:pdq=514:if_outlier_treat=N:Time Taken=15.465070962905884 seconds
rmse_mape_60:nameplate=Cat11:variable=leads:pdq=514:if_outlier_treat=N:Time Taken=1.5651428699493408 seconds

Time Taken RMSE/MAPE: 105.12378525733948 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [86]:
df_rmse_mape.to_csv('MAPE_RMSE_unpaid_suggested.csv', index = False)

# Forecast

In [87]:
def forecast_func(df_all,df_capped,pdq,var,exog_var,if_outlier_treat,nameplate,known_start_dt,known_end_dt,unknown_end_dt):
    
    start_time = time.time()


    # # Setting Dates based on input

    # last two months + some extra days of 3rd month of known data (if any)
    # currently taking as 60 days before known_end_dt 
    known_last2_start_dt = (datetime.strptime(known_end_dt, '%Y-%m-%d').date() - timedelta(days = 60)).strftime("%Y-%m-%d")


    # this is one day before known_last2_start_dt
    end_dt_before_last2 = (datetime.strptime(known_last2_start_dt, '%Y-%m-%d').date() - timedelta(days = 1)).strftime("%Y-%m-%d")

    # from and to date of range we want to predict
    # generally one day after known_end_dt
    unknown_start_dt = (datetime.strptime(known_end_dt, '%Y-%m-%d').date() + timedelta(days = 1)).strftime("%Y-%m-%d")


    # last 31 days of known data
    # 31 days before known_end_dt
    last31_start_dt = (datetime.strptime(known_end_dt, '%Y-%m-%d').date() - timedelta(days = 30)).strftime("%Y-%m-%d")
    
    # subsetting df_all for only the given nameplate
    df_a = df_all[df_all.Category == nameplate]
    df_c = df_capped[df_capped.Category == nameplate]
    
    
    
    # train on data till 60 days before last day of known data.
    # forecast on last 60 days of known data and calculate RMSE and MAPE
    
    train_all_known_a = df_a[known_start_dt:known_end_dt]
    test_all_unknown_a = df_a[unknown_start_dt:unknown_end_dt]
    
    train_all_known_c = df_c[known_start_dt:known_end_dt]
    
    
    # ARIMAX Forecasting
    
    #----------creating pdq = decided pdq
    try:
        # setting p d q values
        p = int(pdq[0])
        d = int(pdq[1])
        q = int(pdq[2])
        
        # deciding training data
        if if_outlier_treat == 'Y':
            train_all_known = train_all_known_c.copy()
        elif if_outlier_treat == 'N':
            train_all_known = train_all_known_a.copy()
        else:
            print(sys._getframe(  ).f_code.co_name+':error=if_outlier_treat flag wrong: if_outlier_treat= '+if_outlier_treat)
        
        # setting test data    
        test_all_unknown = test_all_unknown_a.copy()

        ar = ARIMA(endog=train_all_known[var], exog=train_all_known[exog_var], order=(p,d,q))
        # fit ARIMA model
        model = ar.fit()
        # predict test using ARIMA model
        pred_unknown = model.forecast(steps=len(test_all_unknown), exog=test_all_unknown[exog_var])
        
        # this is because we want actual data in forecasts and not outlier capped data
        train_all_known = train_all_known_a.copy()
        # taking out the forecast in csv file
        df_actual = pd.DataFrame()
        df_actual['date'] = pd.Series(train_all_known.index)
        df_actual[var+'_actual'] = list(train_all_known[var])
        
        df_pred = pd.DataFrame()
        df_pred['date'] = alldates(unknown_start_dt,unknown_end_dt)
        df_pred.date = pd.to_datetime(df_pred.date)
        df_pred[var+'_forecast'] = pd.DataFrame(pred_unknown[0], columns = [var])
        
        ar_forecast = pd.DataFrame()
        ar_forecast = df_actual.append(df_pred)

    except Exception as e:
        # handling if error in given pdq
        ar_forecast = pd.DataFrame(columns = ['date',var+'_actual',var+'_forecast'])
        
        print(sys._getframe(  ).f_code.co_name+':nameplate='+nameplate+':variable='+var+':pdq='+pdq+':if_outlier_treat='+if_outlier_treat+':error='+str(e))
      

    
    
    
    end_time = time.time()
    print(sys._getframe(  ).f_code.co_name+':nameplate='+nameplate+':variable='+var+':pdq='+pdq+':if_outlier_treat='+if_outlier_treat+':Time Taken={0} seconds'.format(end_time - start_time))
    
    return ar_forecast

### Forecast for 511 and no outlier treatment

In [88]:
start_time = time.time()

# creating dataframe to hold forecasts
visit_unpaid_forecast = pd.DataFrame()
lfa_unpaid_forecast = pd.DataFrame()
leads_total_forecast = pd.DataFrame()

exog_var = 'total_spend_nobrand_dig'

for nameplate in tqdm(df_all.Category.unique()):
    for var in ['visits','testdrive','leads']:
        
        if var in ['visits','testdrive','leads']:
            exog_var = 'total_spend_nobrand_nodig'            
        elif var == 'leads':
            exog_var == 'total_spend_nobrand'
        else:
            print('error: wrong variable: var='+var)
            
        pdq = '511'
        if_outlier_treat = 'N'
            
        df_forecast = forecast_func(df_all,df_capped,pdq,var,exog_var,if_outlier_treat,nameplate,known_start_dt,known_end_dt,unknown_end_dt)
        
        df_forecast['Nameplate'] = nameplate
        
        if var == 'visits':
            visit_unpaid_forecast = visit_unpaid_forecast.append(df_forecast)
        elif var == 'testdrive':
            lfa_unpaid_forecast = lfa_unpaid_forecast.append(df_forecast)
        elif var == 'leads':
            leads_total_forecast = leads_total_forecast.append(df_forecast)
        else:
            print('error: wrong forecast variable='+var)
    
    
end_time = time.time()
print("Time Taken RMSE/MAPE: {0} seconds".format(end_time - start_time))

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


forecast_func:nameplate=Cat1:variable=visits:pdq=511:if_outlier_treat=N:Time Taken=1.741288423538208 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


forecast_func:nameplate=Cat1:variable=testdrive:pdq=511:if_outlier_treat=N:Time Taken=1.6489262580871582 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr

...

forecast_func:nameplate=Cat11:variable=visits:pdq=511:if_outlier_treat=N:Time Taken=1.378826379776001 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


forecast_func:nameplate=Cat11:variable=testdrive:pdq=511:if_outlier_treat=N:Time Taken=3.4763741493225098 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


forecast_func:nameplate=Cat11:variable=leads:pdq=511:if_outlier_treat=N:Time Taken=0.8970541954040527 seconds

Time Taken RMSE/MAPE: 66.32451176643372 seconds


In [90]:
visit_unpaid_forecast.to_csv('visit_unpaid_forecast_511_noOUTLIERtreat.csv', index = False)
lfa_unpaid_forecast.to_csv('lfa_unpaid_forecast_511_noOUTLIERtreat.csv', index = False)
leads_total_forecast.to_csv('leads_total_forecast_511_noOUTLIERtreat.csv', index = False)

In [91]:
visit_unpaid_forecast.shape

(9702, 4)

### Forecast for suggested pdqs and outlier treatment

In [93]:
start_time = time.time()

# reading finally decided pdq and outlier treat
#pdq_map_vu = pd.read_excel(pdq_visit_unpaid_decided)
pdq_map_vu = pd.read_csv(pdq_visit_unpaid_decided)


pdq_map_vu['Nameplate'] = pdq_map_vu['Nameplate'].astype(str)
#pdq_map_l = pd.read_excel(pdq_leads_decided)
pdq_map_l = pd.read_csv(pdq_leads_decided)
pdq_map_l['Nameplate'] = pdq_map_l['Nameplate'].astype(str)

# creating dataframe to hold forecasts
visit_unpaid_forecast = pd.DataFrame()
lfa_unpaid_forecast = pd.DataFrame()
leads_total_forecast = pd.DataFrame()

# reading finally decided pdq and outlier treat
#pdq_map_vu = pd.read_excel('Final_Visits_PDQ.xlsx')
pdq_map_vu = pd.read_csv("https://raw.githubusercontent.com/datacoe-publicissapient/risingai2020/master/data/Final_Visits_PDQ.csv")
pdq_map_vu['Nameplate'] = pdq_map_vu['Nameplate'].astype(str)

#pdq_map_l = pd.read_excel('Final_Leads_PDQ.xlsx')
pdq_map_l = pd.read_csv("https://raw.githubusercontent.com/datacoe-publicissapient/risingai2020/master/data/Final_Leads_PDQ.csv")
pdq_map_l['Nameplate'] = pdq_map_l['Nameplate'].astype(str)

exog_var = 'total_spend_nobrand_dig'

for nameplate in tqdm(df_all.Category.unique()):
#for nameplate in ['CHARGER']:
        
    for var in ['visits','testdrive','leads']:
        
        if var in ['visits','testdrive','leads']:
            exog_var = 'total_spend_nobrand_nodig'
            
            pdq_map_vu_nameplate = pdq_map_vu[pdq_map_vu.Nameplate == nameplate]
            p = str(pdq_map_vu_nameplate['p'].values[0])
            d = str(pdq_map_vu_nameplate['d'].values[0])
            q = str(pdq_map_vu_nameplate['q'].values[0])
            pdq = p+d+q
            if_outlier_treat = str(pdq_map_vu_nameplate['outlier_treatment'].values[0])
            
        elif var == 'leads':
            exog_var == 'total_spend_nobrand'
            
            pdq_map_l_nameplate = pdq_map_l[pdq_map_l.Nameplate == nameplate]
            p = str(pdq_map_l_nameplate['p'].values[0])
            d = str(pdq_map_l_nameplate['d'].values[0])
            q = str(pdq_map_l_nameplate['q'].values[0])
            pdq = p+d+q
            if_outlier_treat = str(pdq_map_l_nameplate['outlier_treatment'].values[0])
            
        else:
            print('error: wrong variable: var='+var)
            
            
        df_forecast = forecast_func(df_all,df_capped,pdq,var,exog_var,if_outlier_treat,nameplate,known_start_dt,known_end_dt,unknown_end_dt)
        
        df_forecast['Nameplate'] = nameplate
        
        if var == 'visits':
            visit_unpaid_forecast = visit_unpaid_forecast.append(df_forecast)
        elif var == 'testdrive':
            lfa_unpaid_forecast = lfa_unpaid_forecast.append(df_forecast)
        elif var == 'leads':
            leads_total_forecast = leads_total_forecast.append(df_forecast)
        else:
            print('error: wrong forecast variable='+var)
    
    
end_time = time.time()
print("Time Taken RMSE/MAPE: {0} seconds".format(end_time - start_time))

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


forecast_func:nameplate=Cat1:variable=visits:pdq=514:if_outlier_treat=N:Time Taken=10.5325608253479 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr

forecast_func:nameplate=Cat1:variable=testdrive:pdq=514:if_outlier_treat=N:Time Taken=11.28022289276123 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/tsatools.py:668: RuntimeWarning: overflow encountered in exp
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/tsatools.py:668: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/tsatools.py:669: RuntimeWarning: overflow encountered in exp
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/tsatools.py:669: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()


...

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


forecast_func:nameplate=Cat11:variable=visits:pdq=514:if_outlier_treat=N:Time Taken=15.180858850479126 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr

forecast_func:nameplate=Cat11:variable=testdrive:pdq=514:if_outlier_treat=N:Time Taken=5.616007566452026 seconds
forecast_func:nameplate=Cat11:variable=leads:pdq=514:if_outlier_treat=N:Time Taken=1.034029245376587 seconds

Time Taken RMSE/MAPE: 123.19552731513977 seconds


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [94]:
visit_unpaid_forecast.to_csv('visit_unpaid_forecast_suggested.csv', index = False)
lfa_unpaid_forecast.to_csv('lfa_unpaid_forecast_suggested.csv', index = False)
leads_total_forecast.to_csv('leads_total_forecast_suggested.csv', index = False)

In [98]:
leads_total_forecast.shape

(9702, 4)

In [99]:
files.download('leads_total_forecast_suggested.csv')
#files.download('lfa_unpaid_forecast_suggested.csv')
#files.download('visit_unpaid_forecast_suggested.csv')
#visit_unpaid_forecast_511_noOUTLIERtreat.csv
#lfa_unpaid_forecast_511_noOUTLIERtreat.csv
#leads_total_forecast_511_noOUTLIERtreat.csv

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>